#**Práctica 3: Problemas de Regresión**

Curso: Inteligencia Artificial para Ingenieros

Prof. Carlos Toro N. (carlos.toro.ing@gmail.com)

2022

**Introducción**

El **análisis de regresión** trata de explicar las relaciones entre variables, es uno de los problemas que pueden resolver las técnicas de machine learning. Consiste en encontrar una función $f$ tal que:

$$
\begin{align}y = f(x,\theta), f: R^n\rightarrow R^m\end{align}
$$

$y$ es la variable objetivo de tipo continua, en el caso más típico es un escalar ($m=1$) pero también puede ser más de una variable de respuesta (un vector), $x$ es la variable predictora o de características y $\theta$ son los parámetros a optimizar mediante el proceso de entrenamiento.

La función $f$ puede ser tan simple como una linea recta o algo tan complejo como una red neuronal artificial, la elección dependerá de la complejidad del problema.

Notar que algunas de las técnicas tradicionales como la regresión lineal simple y lineal multivariada han sido históricamente desarrolladas por la estadística, asique es una buena forma de profundizar el tema buscando referencias en esa área.


**En esta práctica veremos:**

1. Regresión lineal simple
2. Regresión polinomial
3. Regresión lineal multivariada
4. Ajuste de curvas con funciones no lineales
5. Ejercicios

En esta práctica usaremos herramientas como Sci-kit learn y Scipy para estudiar este tipo de problemas.

##1. Regresión lineal simple

En la regresión lineal asumimos que la variable dependiente puede ser, aproximadamente, expresada como una combinación lineal de las variables predictoras o características (features). En el caso más simple, una variable predictora, ajustaremos una linea recta a los dadtos. El modleo en este caso tiene la siguiente forma:
$$y = ax+b$$
aquí $a$ es conocida como la *pendiente*, y $b$ como el *intercepto*.

Primero importemos las librerías importantes para el desarrollo de la práctica:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

###Ejemplo 1. Regresión lineal simple con datos simulados

Consideremos los siguientes datos simulados con la relación $y = 2x -5$:

In [ ]:
# simulamos algunos datos con tendencia lineal y con algo de ruido gaussiano
np.random.seed(10)# para asegurar reproducibilidad
Np  = 20 # número de puntos a simular
x   = np.linspace(0,10,Np)
x   = x[:,np.newaxis]# para agregar una dimensión a los datos, los modelos de sklearn requieren datos en la forma [n_samples,n_features]
y   = 2 * x - 5 + 2*np.random.randn(Np,1)# desviación estándar 2 del ruido

In [ ]:
# comprobemos las dimensiones de los datos
print("El arreglo x de la variable independiente tiene dimensión:",x.shape)
print("El arreglo y de la variable dependiente tiene dimensión",y.shape)

Luego, usaremos el modelo ```LinearRegression()``` de Scikit-learn para ajustar los datos y construir la mejor linea posible:

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)# generamos una instancia del modelo, el valor de fit_intercept es por defecto True,
                                            # lo dejamos para recordar que también se puede desacartar dejandolo en False.

model.fit(x, y) # Entrenamos el modelo, ojo, se agrega una dimensión al arreglo x para que sea un vector columna

xfit = np.linspace(0, 10, 1000)          # simulamos algunos datos para visualizar el modelo
yfit = model.predict(xfit[:, np.newaxis])# agregamos un nuevo eje para que la dimensión sea de (1000,1) un vector columna

plt.scatter(x,y)
plt.plot(xfit, yfit,'black')
plt.xlabel('x'); plt.ylabel('y')
plt.legend(["Predicciones","Datos originales"])
#graficamos segmentos de recta que indique el error entre la predicción y el valor real
plt.plot(np.hstack([x,x]).T, np.hstack([y, model.predict(x)]).T, color="red");


La regresión lineal trata de minimizar la suma de los errores al cuadrado $\sum_i (y[i] - \hat{y}[i])^2$; esta es la suma de la longitud de los segmentos rojos en el gráfico de arriba. Los valores estimados $\hat{y}[i]$ son denotados por `yfit` en el código anterior.

La pendiente y el intercepto los podemos obtener desde el modelo definido y previamente ajustado con el método ```fit```

In [ ]:
print("Pendiente del modelo:", model.coef_[0])
print("Intercepto:", model.intercept_)

En este ejemplo, el coeficiente `model.coef_[0]` es la pendiente de la línea ajustada, y el intercepto `model.intercept_` es el punto donde la linea intersecta el eje-y. En scikit leran los parámetros aprendidos de este modelo u otro siempre tienen un guión bajo al final para indetificarlos fácilemnte.

Notemos que los parámetros estimados por el modelo de regresión estuvieron bastante cerca de los valores reales de la recta simulada. Experimentar con el número de datos y desviación estándar de los errores introducidos y evaluar los resultados.

Por otro lado, las métricas de ajuste para evaluar el desempeño del modelo las podemos obtener con la función `mean_squared_error` y `r2_score` desde el paquete `sklearn.metrics`:

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
print("El error MSE del modelo es: ", mean_squared_error(y, model.predict(x)))
print("El valor del coeficiente R2 es: ", r2_score(y, model.predict(x)))

###Ejemplo 2. Regresión lineal simple con datos reales

Predicción del salario anual de una persona en relación a sus años de experiencia. En este ejemplo, volveremos al caso pendiente sobre regresión lineal univariada, agregando división del dataset en un set de entrenamiento y un set de prueba para evaluar nuestro modelo entrenado.

**A. Importamos paquetes de interés**

In [ ]:
# sklearn - Machine Learning tradicional
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

**B. Cargamos los datos**

Antes de ejecutar la siguiente linea, subir el archivo a colab con alguno de los métodos descritos, dejarlo en la misma carpeta del notebook si se trabaja de forma local o indicar la dirección completa.

In [ ]:
dataset = pd.read_csv('Salary - Salary.csv')
dataset.head()

In [ ]:
# Grafiquemos los datos para tener una vista de la relación que hay entre las variables
plt.scatter(dataset.YearsExperience,dataset.Salary,color='red')
plt.xlabel('Experiencia')
plt.ylabel('Salario')
plt.show()

**C. Extraer las variables de interés y guardarlas en numpy arrays:**


Recordar que con pandas, podemos acceder a los valores de las columnas indexando con las posiciones de estas usando el método `iloc` o a través del nombre usando `loc`, generemos entonces nuestras variables x e y

In [ ]:
x = dataset.iloc[:,[0]].values # años de experiencia quedan guardados como numpy array al extraer los valores
y = dataset.loc[:,['Salary']].values # salario anual

print("Tamaño del arreglo x: ",x.size)
print("Forma  del arreglo x: ",x.shape)

**D. Separamos el dataset completo en un set de entrenamiento y uno de prueba.**

Para esto usaremos la función `train_test_split()`, esta recibe como argumentos la variable x e y, la proporción de los datos que serán dejados para prueba o entrenamiento (entre 0 y 1) o el número de muestras para test, además de [otros](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) parámetros.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)#20% de los datos serán dejados para test

**E. Regresión lineal**
* Creamos/instanciamos el modelo
* Entrenamos el modelo con el método ```fit```
* Hacemos predicciones sobre el conjunto de test con método predict
* Calculamos métricas de desempeño en train y test

In [ ]:
modelo_lineal = LinearRegression()#creamos modelo
modelo_lineal.fit(x_train,y_train)#entrenamiento

# evaluamos en train
y_pred_train= modelo_lineal.predict(x_train)
# evaluación en test
y_pred_test = modelo_lineal.predict(x_test)


# calculamos las métricas de desempeño y las guardamos en un dataframe para evaluación o exportar resumen
MSE_train = mean_squared_error(y_train, y_pred_train )
MSE_test  = mean_squared_error(y_test, y_pred_test)
R2_train  = r2_score(y_train, y_pred_train)
R2_test   = r2_score(y_test, y_pred_test)

# dataframe conteniendo las métricas (solo para ordenarlos)
dmetricas = {'Train': [MSE_train, R2_train], 'Test': [MSE_test, R2_test]}# datos resumen
Resumendf = pd.DataFrame(dmetricas,index = ['MSE','R2_score'])
Resumendf

**F. Visualización de resultados**

In [ ]:
plt.figure(figsize = (7,7))
plt.plot(x_train,y_train,'or',x_test,y_test,'og',x_train,y_pred_train,'-b')
plt.legend(['Train','Test'])
plt.xlabel('Experiencia')
plt.ylabel('Salario')
plt.show()

**Observación**: en el ejemplo anterior las métricas de desempeño del modelo fueron calculadas en una porción de los datos, donde la separación inicial del conjunto de datos de entrenamiento y prueba fue aleatoria, hubiesen cambiado las métricas si tenemos una separación diferente? Si! qué hacemos entonces para tener una evaluación del modelo más realista? podemos aplicar la técnicas de **Validación Cruzada**.

##2. Validación Cruzada

En los ejemplos anteriores, la validación más simple de los modelos consistió en:
* dividir el conjunto de datos en dos subconjuntos: uno de entrenamiento y otro de prueba
* ajustar el modelo en el conjunto de entrenamiento
* estimar el error de entrenamiento en el conjunto de entrenamiento
* estimar el error de prueba en el conjunto de prueba.

Sin embargo, para evaluar la robustez del modelo y su capacidad de generalización, no es suficiente con este procedimiento. En particular, si el conjunto de datos es pequeño, el error de prueba será inestable y no reflejará la verdadera tasa de error que podríamos observar con el mismo modelo en un conjunto de datos de prueba de tamaño grande. De hecho, dado que la separación es aleatoria de los datos, puede darse el caso que las muestras de prueba favorezcan (solo por casualidad) la tarea de predicción del modelo.

Para tener entonces una mejor evaluación del desempeño del modelo, aquí nos puede ayudar la técnica de **validación cruzada**.Esta técnica permite evaluar la **robustez** de un modelo predictivo al repetir el proceso de división del conjunto de datos. Esto nos dará muchos errores de entrenamiento y prueba y por consiguiente una estimación de la variabilidad del desempeño del modelo.

Existen diferentes tipos de estrategias de validación cruzada (ej. K-fold cross validation, Leave One Out, Suffle Split, Stratified k-fold, etc.), en este práctico nos enfocaremos en la del tipo "suffle-split" (o división con revolvimiento aleatorio de los datos). En cada iteración de esta estrategia tendremos que:

* revolver aleatoriamente el orden de las muestras de una copia del conjunto de datos completo;
* dividir el conjunto de datos en uno de entrenamiento y otro de prueba;
* entrenar el nuevo modelo en el conjunto de entrenamiento;
* evaluar el error de prueba en el conjunto de prueba.

Este procedimiento se repite `n_splits` veces. Tener presente que estos procedimientos implican evaluar `n_splits`modelos, lo que incrementa el tiempo de computacional.

<center><img src=https://inria.github.io/scikit-learn-mooc/_images/shufflesplit_diagram.png width="850" ></center>


[fuente](https://inria.github.io/scikit-learn-mooc/python_scripts/cross_validation_train_test.html)

En la figura anterior, se muestra el caso particular de la estrategia de validación cruzada **shuffle-split** usando `n_splits = 5`.

Para el Ejemplo 2 del bloque anterior de regresión lineal simple implementemos esta estrategia y evaluemos los resultados.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

regressor = LinearRegression()#creamos modelo

cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
cv_results = cross_validate( regressor, x, y, cv=cv, scoring="neg_mean_absolute_error")

Los resultados `cv_results`son almacenados en un diccionario de python. Lo convertiremos a un `DataFrame`de pandas para visualizarlo y manipularlo más facilmente.

In [ ]:
import pandas as pd

cv_results = pd.DataFrame(cv_results)
cv_results.head()

**NOTAS**

Un **score** es una métrica para la cual valores más grandes significan mejores resultados. Por el contrario, una metrica de error indicará mejores resultados cuando sus valores sean más pequeños. El parámetro `scoring` en `cross_validate` siempre espera una función que es un score.

En sci-kit learn, todas las métricas de error como la de `mean_absolute_error`, pueden ser transformadas a score para ser usadas en `cross_validate` por medio de anteponer el string `neg_` antes del nombre, com en este ejemplo `scoring = "neg_mean_absolute_error"`.

En nuestro caso, revirtamos la negación para obtener el valor del error correspondiente y observemos algunos de los valores:

In [ ]:
cv_results["test_error"] = -cv_results["test_score"]
cv_results.head()

Resumamos los resultados en términos de un error de prueba medio y su variabilidad indicada por la desviación estándar:

In [ ]:
print(f"El promedio de el error de prueba usando validación cruzada es: "
      f"{cv_results['test_error'].mean():.2f} $")

In [ ]:
print(f"La desviación estándar del error de prueba es: "
      f"{cv_results['test_error'].std():.2f} $")

Estas métricas serán mas realistas para reportar el desempeño del modelo. Finalmente, si queremos hacer predicciones sobre datos nuevos, con qué modelo lo hacemos? En este caso se sugiere entrenar el modelo seleccionado (en este caso solo tenemos uno de tipo lineal) en todo el conjunto de datos original, teniendo en cuenta que las métricas de desempeño son las calculadas con la estrategia anterior.

Finalmente, para seleccionar un modelo cuando estamos analizando varias alternativas, podemos hacerlo comparando sus métricas usando la estrategia descrita. Notar que tanto el valor promedio de las métricas o scores y la desviación estándar son importantes. Si dos modelos entregan métricas promedio similares, se prefiere el que tenga una desviación estándar menor.

**Preguntas adicionales**
- ¿Podemos obtener más de una métrica usando `cross_validate`? por ejemplo, el coeficiente $R^2$, y las métricas obtenidas en los conjuntos de entrenamiento. Buscar en la documentación e implementarlo
- Sci-kit learn también implementa validación cruzada del tipo [K-Fold](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators), al respecto, mencione alguna ventaja de la estrategia implementada en esta práctica sobre la del tipo K-Fold.

##3. Regresión polinomial

En este caso nos interesa ajustar un polinomio de grado n a los datos. El polinomio tiene la siguiente forma:
$$
y = a_0 + a_1*x + a_2*x^2+...+a_n*x^n
$$

Primero simulamos el conjunto de datos:

In [ ]:
Np = 100 # número de datos a simular
X  = 8*np.random.rand(Np,1)-4
y  = 0.5*X**2 + X + 2 + np.random.randn(Np,1)
plt.scatter(X, y);

Claramente los datos no pueden ser ajustados con una linea recta, en este caso, necesitaremos ajustar una parábola o polinomio de grado 2. Para esto usaremos el `PolynomialFeatures()` para incluir estas transformaciones polinómicas sobre las características y así poder ocupar el modelo de regresión lineal (en los parámetros) para estimar los coeficientes del polinomio.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree = 2, include_bias=False)    # Definimos la transformación
X_poly        = poly_features.fit_transform(X)                        # Aplicamos la transformación

In [ ]:
print("Primer dato original: ",X[0])
print("Primer dato transformado: ",X_poly[0])

Ahora, además de incluir el dato original x se incluye su valor al cuadrado. Ahora ajustemos los datos usando el modelo de regresión lineal  `LinearRegression()`

In [ ]:
pol_reg = LinearRegression()
pol_reg.fit(X_poly,y)
print('Intercepto estimado: ',pol_reg.intercept_)
print('Coeficientes polinomio: ',pol_reg.coef_)

Visualizamos los resultados:

In [ ]:
xfit = np.linspace(-4, 4, 1000)
yfit = pol_reg.predict(poly_features.fit_transform(xfit[:, np.newaxis]))

plt.scatter(X, y)
plt.plot(xfit, yfit,'r');
plt.xlabel('x')
plt.ylabel('y')
plt.legend(["Predicciones","Datos originales"])
plt.show()

##4. Regresión lineal multivariada

Antes de resolver el problema, es importante recordar que muchos de los modelos que usaremos, tanto supervisados como no supervisados, requieren que los datos tanto en la matriz de características como la variable objetivo, se ordenen de cierta forma, en particular como se muestra en la siguiente figura:

<center>
    <img width="60%" src="https://jakevdp.github.io/PythonDataScienceHandbook/figures/05.02-samples-features.png">
</center>


[fuente](https://jakevdp.github.io/PythonDataScienceHandbook/05.02-introducing-scikit-learn.html)


**A. Dataset**

Los datos que usaremos corresponden a datos de precio de propiedades de California, el *California housing dataset* disponibles en `sklearn.datasets`.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing(as_frame=True)
data, target = housing.data, housing.target

En este conjunto de datos el objetivo es predecir la mediana del valor de casa en un área de California. Las características (features) coleccionados se basaron en información geográfica e inmobiliarios en general.

Veamos algunos detalles del conjunto de datos:

In [ ]:
print(housing.DESCR)

In [ ]:
data.head()

Para simplificar las visualizaciones futuras, transformemos los precios del rango 100(k\$) al de miles de dólares (k\$)

In [ ]:
target *= 100
target.head()

**B. Separación de datos y entrenamiento**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Dividimos el dataset entre el conjunto de entrenamiento y prueba
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.3)

modelo_lineal_MV = LinearRegression()# creamos instancia del modelo
modelo_lineal_MV.fit(data_train,target_train)# entrenamiento

**C. Probamos en conjunto de test y evaluamos algunas métricas**

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

# evaluación en train
target_pred_train= modelo_lineal_MV.predict(data_train)
# evaluación en test
target_pred_test = modelo_lineal_MV.predict(data_test)


# cálculo de métricas de desempeño y las guardamos en un dataframe para evaluación o exportar resumen
MSE_train = mean_squared_error(target_train, target_pred_train )
MSE_test  = mean_squared_error(target_test, target_pred_test)
R2_train  = r2_score(target_train,target_pred_train)
R2_test   = r2_score(target_test, target_pred_test)

# dataframe conteniendo las métricas
dmetricas = {'Train': [MSE_train, R2_train], 'Test': [MSE_test, R2_test]}# datos resumen
Resumendf = pd.DataFrame(dmetricas,index = ['MSE','R2_score'])
Resumendf


**D. Visualizamos los resultados**

In [ ]:
plt.figure(figsize = (7,7))
plt.scatter(target_test,target_pred_test, color = 'red')
plt.xlabel('Valor de prueba real')
plt.ylabel('Valor de prueba predicho')
plt.show()

El gráfico anterior nos dice que el modelo no tiene la capacidad suficiente para representar correctamente la relación entre los atributos y la variable objetivo a predecir, si fuese un modelo adecuado, los puntos en el gráfico de dispersión anterior tenderían a quedar en una linea recta.

### Mejorando los resultados

Para mejorar los resultados tenemos varias opciones, por ejemplo:
* Cambiar el tipo de modelo de regresión (existen varios disponibles en Sci-kit learn)
* Incluir nuevas variables transformadas (ej. agregando factores polinomiales) o preprocesamiento previo (ej. estandarizado de los datos).

* Incluir técnicas de regularización a los modelos que lo permitan, etc.

En el siguiente ejemplo, las predicciones se harán implementando un modelo lineal en los coeficientes que además incluya interacciones entre las variables, ej. x1*x2, x1*x3,...

In [ ]:
# Creamos nuevas variables a partir de los features originales, para incluir interacciones
from sklearn.preprocessing import PolynomialFeatures
inter = PolynomialFeatures(interaction_only=True,include_bias = False)# solo considera interacciones, no potencias
Xinter_train = inter.fit_transform(data_train)
Xinter_test  = inter.transform(data_test)

In [ ]:
# ojo, que ahora el número de características aumentó al incluir las interacciones
print("Forma original del conjunto de entrenamiento: ",data_train.shape)
print("Nueva forma del conjunto de entrenamiento: ",Xinter_train.shape)

In [ ]:
modelo_lineal_Inter = LinearRegression()# creamos modelo para incluir interacciones
modelo_lineal_Inter.fit(Xinter_train,target_train)# entrenamiento

# evaluación en test
y_pred_test  = modelo_lineal_Inter.predict(Xinter_test)
y_pred_train = modelo_lineal_Inter.predict(Xinter_train)

# cálculo de métricas de desempeño y las guardamos en un dataframe para evaluación o exportar resumen
MSE_train = mean_squared_error(target_train, y_pred_train)
MSE_test  = mean_squared_error(target_test, y_pred_test)
R2_train  = r2_score(target_train, y_pred_train)
R2_test   = r2_score(target_test, y_pred_test)

# dataframe conteniendo las métricas
dmetricas = {'Train': [MSE_train, R2_train], 'Test': [MSE_test, R2_test]}# datos resumen
Resumendf = pd.DataFrame(dmetricas,index = ['MSE_inter','R2_score_inter'])
Resumendf

In [ ]:
# Visualización de resultados
plt.figure(figsize = (7,7))
plt.scatter(target_test,y_pred_test, color = 'blue')
plt.xlabel('Valor de prueba real')
plt.ylabel('Valor de prueba predicho con modelo incluyendo interacciones')
plt.show()

Hubo una ligera mejora, pero aún se puede incrementar. Probemos utilizando otros modelos disponibles en `sci-kit learn`. En el siguiente ejemplo implementaremos lo anterior y además implementaremos la estrategia de validación cruzada para tener una mejor idea del comportamiento de los modelos en relación a su capacidad de generalización.

**A. Primero importemos los modelos disponibles**

In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Para ejecutar validación cruzada
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

# Diferentes modelos disponibles en sci-kit learn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

**B. Definamos algunos ajustes previos al entrenamiento y creemos las instancias de los modelos.**

In [ ]:
# variables de usuario a ajustar
SEED       = 9
FOLDS      = 10
SCORE_REG  = ["neg_mean_squared_error","r2"] #en este caso, un score es una variable cuyo valor mientras mayor sea mejor es el resultado evaluado, acá usamos el negativo del MSE para pasarlo como argumento a la validación cruzada, yaque trabaja con scores.

# hold different regression models in a single dictionary
models = dict()
models["Linear"]        = LinearRegression()
models["Lasso"]         = Lasso()
models["ElasticNet"]    = ElasticNet()
models["Ridge"]         = Ridge()
models["BayesianRidge"] = BayesianRidge()
models["KNN"]           = KNeighborsRegressor()
models["DecisionTree"]  = DecisionTreeRegressor()
models["AdaBoost"]      = AdaBoostRegressor()
models["GradientBoost"] = GradientBoostingRegressor()
models["RandomForest"]  = RandomForestRegressor()

In [ ]:
# 10-fold cross validation para cada modelo
MSE_Test  = list()
MSE_Train = list()
R2_Test   = list()
R2_Train  = list()
model_names   = list()
CV            = ShuffleSplit(n_splits=FOLDS, test_size = 0.3, random_state=SEED)
for model_name in models:
    model   = models[model_name]

    cv_results = cross_validate(model, data, target, cv=CV, scoring=SCORE_REG,return_train_score=True)

    MSE_Test.append(-cv_results["test_neg_mean_squared_error"])# MSE en Test
    MSE_Train.append(-cv_results["train_neg_mean_squared_error"])# MSE en Train
    R2_Test.append(cv_results["test_r2"])# R2 en Test
    R2_Train.append(cv_results["train_r2"])# R2 en Train

    model_names.append(model_name)
    print("MSE en TEST modelo {:>20}: {:.2f}, {:.2f}".format(model_name, round( -cv_results["test_neg_mean_squared_error"].mean(), 3), round(cv_results["test_neg_mean_squared_error"].std(), 3)))


In [ ]:
# Gráficos de caja para comparar el resultado del MSE de los modelos en TEST
figure = plt.figure(figsize = (10,6));
ax1 = figure.add_subplot(111);
plt.boxplot(MSE_Test);
ax1.set_xticklabels(model_names, rotation = 45, ha="right");
ax1.set_ylabel("Error cuadrático medio (MSE)");
ax1.set_title('Comparación de modelos de regresión en TEST')
plt.margins(0.05, 0.1);
plt.show();


In [ ]:
# Gráficos de caja para comparar el resultado del MSE de los modelos en TRAIN
figure = plt.figure(figsize = (10,6));
ax2 = figure.add_subplot(111);
plt.boxplot(MSE_Train);
ax2.set_xticklabels(model_names, rotation = 45, ha="right");
ax2.set_ylabel("Error cuadrático medio (MSE)");
ax2.set_title('Comparación de modelos de regresión en TRAIN')
plt.margins(0.05, 0.1);
plt.show();

¿Qué puede concluir de los resultados anteriores?


##4. Ajuste de curvas con funciones no lineales

A diferencia de los problemas anteriores, en este caso al menos uno de los parámetros a estimar no está combinado linealmente con los datos, i.e. puede ser parte del exponente en una función exponencial, o el argumento de una función trigonométrica, etc.

Estos problemas generalmente aparecen en análisis de datos experimentales donde los modelos son conocidos y contienen no linealidades, ej:

* Modelo de la corriente en un circuito alterno: $I(t)=Asin(2\pi f\cdot t+\phi)
+\epsilon$

* Modelo de la corriente de un diodo: $I_D = I_S\cdot\left(e^{\frac{V_D}{nV_T}}-1\right)$

* Respuesta temporal de un sistema de segundo orden sub-amortiguado ante entrada escalón: $y(t) = 1 - \frac{e^{-\xi\omega_n t}}{\sqrt{(1-\xi^2)}}\cdot sin\left(\omega_n\sqrt{(1-\xi^2)}\cdot t+\phi   \right)$


###Ejemplo 1. Identificación de un sistema dinámico de primer orden

En este ejemplo estudiaremos la respuesta de un sistema dinámico de primer orden ante una entrada de tipo escalón unitario (función de Heaviside con amplitud 1). El modelo que rige la respuesta (salida) del sistema ante este tipo de estímulo es bien conocido y tiene la siguiente forma:

$$
y(t) = k (1-e^{-t/\tau}), t\geq0
$$

donde:

$y$: es la respuesta del sistema

$k$: es la ganancia del sistema

$t$: es el tiempo en seg

$\tau$: es la constante de tiempo, i.e. tiempo para el cual la respuesta alcanza $\approx 63\%$ del valor final.

Partamos por simular unos datos asumiendo algunos valores de los parámetros de interés del sistema, ej. $k=2.5$ y $\tau=1.0$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(10)# para asegurar reproducibilidad
Np      = 100 # número de puntos a simular
k_real  = 2.5
tau_real= 1.0
t       = np.linspace(0,10,Np)
y       = k_real*(1-np.exp(-t/tau_real)) + 0.1*np.random.randn(Np)# añadimos ruido para hacer más realista la simulación

# graficamos los datos
plt.plot(t,y,linewidth=1.5)
plt.xlabel('Tiempo (seg)'); plt.ylabel('Amplitud (u.a.)')
plt.axis([0, t[-1], 0, 3])
plt.show()

Para ajustar el modelo a los datos simulados usaremos la libería SciPy.

In [ ]:
from scipy.optimize import curve_fit

#1. Primero definimos el modelo que rige a los datos
def fun(t,k,tau):
    return k*(1-np.exp(-t/tau))

#2. Ajustamos el modelo a los datos
init_values   = [2,0.5]#valores iniciales, suposición según conocimiento experto, pueden no incluirse
best_fit, cov = curve_fit(fun,t,y,p0=init_values)

print(f"Los parámetros estimados son k:{best_fit[0]} y tau:{best_fit[1]}")


In [ ]:
#3. graficamos los datos
plt.figure(figsize=(8,5))
plt.plot(t,y,'-b',t,fun(t,best_fit[0],best_fit[1]),'--r',linewidth=1.5)
plt.xlabel('Tiempo (seg)'); plt.ylabel('Amplitud (u.a.)')
plt.axis([0, t[-1], 0, 3])
plt.legend(['Datos','Ajuste'])
plt.show()

## 5. Ejercicios

**Ejercicio 1. Regresión lineal con una variable independiente**

En este ejercicio usaremos el conjunto de datos adjunto: *penguins_regression.csv* para predecir el peso de pingüinos en función de la longitud de su aleta.

Para esto se pide:
1. Cargar los datos y explorarlos.
2. Dividir el conjunto de datos en uno de entrenamiento y otro de prueba con `train_test_split()`
3. Crear el modelo de regresión con el objeto `LinearRegression()` de `sci-kit learn`
4. Entrenar el modelo
5. Indicar los valores de la pendiente y coeficiente de posición
6. Graficar los datos en un gráfico de dispersión e incluir una simulación del modelo sobre estos (la linea recta con los parámetros encontrados)
7. Calcular las métricas de regresión (MSE, RMSE y $R^2$) sobre ambos conjuntos de datos y comentarlas. Qué ventaja tiene la métrica RMSE sobre el MSE?
8. Si entrenamos el mismo modelo usando validación cruzada, cómo cambian las métricas de desempeño para modelo?

In [ ]:
# Cargamos los datos
pinguinos    = pd.read_csv("penguins_regression.csv")# primero subirlos a colab
feature_name = "Longitud Aleta (mm)"
target_name  = "Masa Corporal (g)"
data, target = pinguinos[[feature_name]], pinguinos[target_name]

In [ ]:
# Código aquí

**Ejercicio 2. Regresión multivariada**

Usando validación cruzada, seleccionar el mejor modelo para los datos de estimación de fuerza de compresión de concreto estudiados en el práctico de Pandas.

Realizar además un gráfico de dispersión entre los valores predichos y valores reales de la variable target para el mejor modelo estimado y reportar las métricas medias junto a su desviación estándar para dicho modelo. Justifique su selección.

In [ ]:
# Código aquí

##Referencias

- Parte de los ejemplos fueron adaptados del MOOC de Sci-kit learn: https://www.fun-mooc.fr/en/courses/machine-learning-python-scikit-learn/